In [1]:
!pip install --upgrade pandas-profiling
!pip install --upgrade hypertools
!pip install --upgrade pandas

     |████████████████████████████████| 241 kB 2.7 MB/s eta 0:00:01
     |████████████████████████████████| 1.5 MB 5.1 MB/s eta 0:00:01
     |████████████████████████████████| 609 kB 15.8 MB/s eta 0:00:01
     |████████████████████████████████| 58 kB 3.5 MB/s  eta 0:00:01
     |████████████████████████████████| 12.4 MB 17.9 MB/s eta 0:00:01


  Created wheel for tangled-up-in-unicode: filename=tangled_up_in_unicode-0.0.4-py3-none-any.whl size=1545721 sha256=e74f0445343fc9dcab8269f2ace66c9d43b1e9fca1014a96c00bdbaad6328581
  Stored in directory: /root/.cache/pip/wheels/22/bb/74/1c79b7768407fd0acee5a61216474bd95b05ccdd38eac9c992
Successfully built tangled-up-in-unicode
ERROR: hypertools 0.6.2 has requirement scikit-learn<0.22,>=0.19.1, but you'll have scikit-learn 0.22.2.post1 which is incompatible.
ERROR: datalab 1.1.5 has requirement pandas-profiling==1.4.0, but you'll have pandas-profiling 2.6.0 which is incompatible.
ERROR: allennlp 0.9.0 has requirement spacy<2.2,>=2.1.0, but you'll have spacy 2.2.3 which is incompatible.
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.1.3
    Uninstalling matplotlib-3.1.3:
      Successfully uninstalled matplotlib-3.1.3
  Attempting uninstall: phik
    Found existing installation: phik 0.9.8
    Uninstalling phik-0.9.8:
      Successfully uninstalled phik

ERROR: tpot 0.11.1 has requirement scikit-learn>=0.22.0, but you'll have scikit-learn 0.21.3 which is incompatible.
ERROR: kmeans-smote 0.1.2 has requirement imbalanced-learn<0.5,>=0.4.0, but you'll have imbalanced-learn 0.6.2 which is incompatible.
ERROR: kmeans-smote 0.1.2 has requirement numpy<1.16,>=1.13, but you'll have numpy 1.18.1 which is incompatible.
ERROR: kmeans-smote 0.1.2 has requirement scikit-learn<0.21,>=0.19.0, but you'll have scikit-learn 0.21.3 which is incompatible.
ERROR: imbalanced-learn 0.6.2 has requirement scikit-learn>=0.22, but you'll have scikit-learn 0.21.3 which is incompatible.
ERROR: datalab 1.1.5 has requirement pandas-profiling==1.4.0, but you'll have pandas-profiling 2.6.0 which is incompatible.
ERROR: cesium 0.9.12 has requirement scikit-learn>=0.22.1, but you'll have scikit-learn 0.21.3 which is incompatible.
ERROR: allennlp 0.9.0 has requirement spacy<2.2,>=2.1.0, but you'll have spacy 2.2.3 which is incompatible.
  Attempting uninstall: scikit-le

In [2]:
import os
import re
import gensim

import numpy as np 
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="whitegrid")

from textblob import TextBlob
from gensim.models.ldamodel import LdaModel
from gensim.corpora.dictionary import Dictionary

from nltk import tokenize
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.corpus import wordnet
from bs4 import BeautifulSoup
from gensim import matutils

import nltk
nltk.download('wordnet')

from sklearn.feature_extraction.text import CountVectorizer

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
base_path = "/kaggle"

# Chech if its in kaggle environment 
if os.path.exists(base_path):
    input_path = os.path.join(base_path, "input", "nlp-getting-started")
    output_path = os.path.join(base_path, "working")
else:
    base_path = "data"
    input_path = base_path
    output_path = os.path.join(base_path, "submissions")


train_file = os.path.join(input_path, "train.csv")
test_file = os.path.join(input_path, "test.csv")

In [4]:
train_df = pd.read_csv(train_file)
test_df = pd.read_csv(test_file)

train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
target_names = ["Not real disaster", "Real disaster"]
real_disaster = 1

## Data cleaning

#### Lemmatize example

In [6]:
WordNetLemmatizer().lemmatize("went", "v")

'go'

Stemming example

In [7]:
stemmer = SnowballStemmer("english")
list(map(stemmer.stem, ["houses", "feets", "avengers"]))

['hous', 'feet', 'aveng']

POS tagging

In [8]:
sentence1 = ["I", "played", "football", "on", "Anfield", "with", "Gerard"]
sentence2 = ["I", "did", "not", "buy", "a", "cd", "from", "ACDC"]

In [9]:
print(nltk.pos_tag(sentence1))
print(nltk.pos_tag(sentence2))

[('I', 'PRP'), ('played', 'VBD'), ('football', 'NN'), ('on', 'IN'), ('Anfield', 'NNP'), ('with', 'IN'), ('Gerard', 'NNP')]
[('I', 'PRP'), ('did', 'VBD'), ('not', 'RB'), ('buy', 'VB'), ('a', 'DT'), ('cd', 'NN'), ('from', 'IN'), ('ACDC', 'NNP')]


Lemmatize + POS Tag

In [10]:
def get_wordnet_pos(sentence):
    """Map POS tag to first character lemmatize() accepts"""
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    
    tagging = nltk.pos_tag(sentence)
    result = [tag_dict.get(tag[0], wordnet.NOUN) for _, tag in tagging]

    return result

In [11]:
tags1 = get_wordnet_pos(sentence1)
tags2 = get_wordnet_pos(sentence2)

lemmatizer = WordNetLemmatizer()

print([lemmatizer.lemmatize(word, tag) for word, tag in zip(sentence1, tags1)])
print([lemmatizer.lemmatize(word, tag) for word, tag in zip(sentence2, tags2)])

['I', 'play', 'football', 'on', 'Anfield', 'with', 'Gerard']
['I', 'do', 'not', 'buy', 'a', 'cd', 'from', 'ACDC']


TextBlob Lemmatizer with POS tagging

In [13]:
def lemmatize_with_postag(sentence):
    sent = TextBlob(sentence)
    tag_dict = {"J": 'a', 
                "N": 'n', 
                "V": 'v', 
                "R": 'r'}
    words_and_tags = [(w, tag_dict.get(pos[0], 'n')) for w, pos in sent.tags]    
    lemmatized_list = [wd.lemmatize(tag) for wd, tag in words_and_tags]
    
    return " ".join(lemmatized_list)

# Lemmatize
sentence = "The striped bats are hanging on their feet for best"
print(sentence)
lemmatize_with_postag(sentence)

The striped bats are hanging on their feet for best


'The striped bat be hang on their foot for best'

Apply this to our dataset

In [14]:
def clean_html_encoding(text):
    text = BeautifulSoup(text, 'lxml').get_text()
    return text


def remove_html_tags(text):
    html = re.compile(r'<.*?>')
    return html.sub(r'',text)


def remove_urls(text):
    links_reg = re.compile(r'https?://[A-Za-z0-9./]+')
    return links_reg.sub(r'', text)


def remove_mentions(text):
    mentions_reg = re.compile(r'@[A-Za-z0-9]+')
    return mentions_reg.sub(r'', text)


def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)


def remove_puntuation(tokens):
    return [token for token in tokens if token.isalpha()]

def clean_chars_and_contracts(tweet):
    # Special characters
    tweet = re.sub(r"\x89Û_", "", tweet)
    tweet = re.sub(r"\x89ÛÒ", "", tweet)
    tweet = re.sub(r"\x89ÛÓ", "", tweet)
    tweet = re.sub(r"\x89ÛÏWhen", "When", tweet)
    tweet = re.sub(r"\x89ÛÏ", "", tweet)
    tweet = re.sub(r"China\x89Ûªs", "China's", tweet)
    tweet = re.sub(r"let\x89Ûªs", "let's", tweet)
    tweet = re.sub(r"\x89Û÷", "", tweet)
    tweet = re.sub(r"\x89Ûª", "", tweet)
    tweet = re.sub(r"\x89Û\x9d", "", tweet)
    tweet = re.sub(r"å_", "", tweet)
    tweet = re.sub(r"\x89Û¢", "", tweet)
    tweet = re.sub(r"\x89Û¢åÊ", "", tweet)
    tweet = re.sub(r"fromåÊwounds", "from wounds", tweet)
    tweet = re.sub(r"åÊ", "", tweet)
    tweet = re.sub(r"åÈ", "", tweet)
    tweet = re.sub(r"JapÌ_n", "Japan", tweet)    
    tweet = re.sub(r"Ì©", "e", tweet)
    tweet = re.sub(r"å¨", "", tweet)
    tweet = re.sub(r"SuruÌ¤", "Suruc", tweet)
    tweet = re.sub(r"åÇ", "", tweet)
    tweet = re.sub(r"å£3million", "3 million", tweet)
    tweet = re.sub(r"åÀ", "", tweet)
    
    # Contractions
    tweet = re.sub(r"he's", "he is", tweet)
    tweet = re.sub(r"there's", "there is", tweet)
    tweet = re.sub(r"We're", "We are", tweet)
    tweet = re.sub(r"That's", "That is", tweet)
    tweet = re.sub(r"won't", "will not", tweet)
    tweet = re.sub(r"they're", "they are", tweet)
    tweet = re.sub(r"Can't", "Cannot", tweet)
    tweet = re.sub(r"wasn't", "was not", tweet)
    tweet = re.sub(r"don\x89Ûªt", "do not", tweet)
    tweet = re.sub(r"aren't", "are not", tweet)
    tweet = re.sub(r"isn't", "is not", tweet)
    tweet = re.sub(r"What's", "What is", tweet)
    tweet = re.sub(r"haven't", "have not", tweet)
    tweet = re.sub(r"hasn't", "has not", tweet)
    tweet = re.sub(r"There's", "There is", tweet)
    tweet = re.sub(r"He's", "He is", tweet)
    tweet = re.sub(r"It's", "It is", tweet)
    tweet = re.sub(r"You're", "You are", tweet)
    tweet = re.sub(r"I'M", "I am", tweet)
    tweet = re.sub(r"shouldn't", "should not", tweet)
    tweet = re.sub(r"wouldn't", "would not", tweet)
    tweet = re.sub(r"i'm", "I am", tweet)
    tweet = re.sub(r"I\x89Ûªm", "I am", tweet)
    tweet = re.sub(r"I'm", "I am", tweet)
    tweet = re.sub(r"Isn't", "is not", tweet)
    tweet = re.sub(r"Here's", "Here is", tweet)
    tweet = re.sub(r"you've", "you have", tweet)
    tweet = re.sub(r"you\x89Ûªve", "you have", tweet)
    tweet = re.sub(r"we're", "we are", tweet)
    tweet = re.sub(r"what's", "what is", tweet)
    tweet = re.sub(r"couldn't", "could not", tweet)
    tweet = re.sub(r"we've", "we have", tweet)
    tweet = re.sub(r"it\x89Ûªs", "it is", tweet)
    tweet = re.sub(r"doesn\x89Ûªt", "does not", tweet)
    tweet = re.sub(r"It\x89Ûªs", "It is", tweet)
    tweet = re.sub(r"Here\x89Ûªs", "Here is", tweet)
    tweet = re.sub(r"who's", "who is", tweet)
    tweet = re.sub(r"I\x89Ûªve", "I have", tweet)
    tweet = re.sub(r"y'all", "you all", tweet)
    tweet = re.sub(r"can\x89Ûªt", "cannot", tweet)
    tweet = re.sub(r"would've", "would have", tweet)
    tweet = re.sub(r"it'll", "it will", tweet)
    tweet = re.sub(r"we'll", "we will", tweet)
    tweet = re.sub(r"wouldn\x89Ûªt", "would not", tweet)
    tweet = re.sub(r"We've", "We have", tweet)
    tweet = re.sub(r"he'll", "he will", tweet)
    tweet = re.sub(r"Y'all", "You all", tweet)
    tweet = re.sub(r"Weren't", "Were not", tweet)
    tweet = re.sub(r"Didn't", "Did not", tweet)
    tweet = re.sub(r"they'll", "they will", tweet)
    tweet = re.sub(r"they'd", "they would", tweet)
    tweet = re.sub(r"DON'T", "DO NOT", tweet)
    tweet = re.sub(r"That\x89Ûªs", "That is", tweet)
    tweet = re.sub(r"they've", "they have", tweet)
    tweet = re.sub(r"i'd", "I would", tweet)
    tweet = re.sub(r"should've", "should have", tweet)
    tweet = re.sub(r"You\x89Ûªre", "You are", tweet)
    tweet = re.sub(r"where's", "where is", tweet)
    tweet = re.sub(r"Don\x89Ûªt", "Do not", tweet)
    tweet = re.sub(r"we'd", "we would", tweet)
    tweet = re.sub(r"i'll", "I will", tweet)
    tweet = re.sub(r"weren't", "were not", tweet)
    tweet = re.sub(r"They're", "They are", tweet)
    tweet = re.sub(r"Can\x89Ûªt", "Cannot", tweet)
    tweet = re.sub(r"you\x89Ûªll", "you will", tweet)
    tweet = re.sub(r"I\x89Ûªd", "I would", tweet)
    tweet = re.sub(r"let's", "let us", tweet)
    tweet = re.sub(r"it's", "it is", tweet)
    tweet = re.sub(r"can't", "cannot", tweet)
    tweet = re.sub(r"don't", "do not", tweet)
    tweet = re.sub(r"you're", "you are", tweet)
    tweet = re.sub(r"i've", "I have", tweet)
    tweet = re.sub(r"that's", "that is", tweet)
    tweet = re.sub(r"i'll", "I will", tweet)
    tweet = re.sub(r"doesn't", "does not", tweet)
    tweet = re.sub(r"i'd", "I would", tweet)
    tweet = re.sub(r"didn't", "did not", tweet)
    tweet = re.sub(r"ain't", "am not", tweet)
    tweet = re.sub(r"you'll", "you will", tweet)
    tweet = re.sub(r"I've", "I have", tweet)
    tweet = re.sub(r"Don't", "do not", tweet)
    tweet = re.sub(r"I'll", "I will", tweet)
    tweet = re.sub(r"I'd", "I would", tweet)
    tweet = re.sub(r"Let's", "Let us", tweet)
    tweet = re.sub(r"you'd", "You would", tweet)
    tweet = re.sub(r"It's", "It is", tweet)
    tweet = re.sub(r"Ain't", "am not", tweet)
    tweet = re.sub(r"Haven't", "Have not", tweet)
    tweet = re.sub(r"Could've", "Could have", tweet)
    tweet = re.sub(r"youve", "you have", tweet)  
    tweet = re.sub(r"donå«t", "do not", tweet)   

    return tweet

def clean_text(text):
    text = clean_html_encoding(text)
    text = remove_html_tags(text)
    text = remove_urls(text)
    text = remove_mentions(text)
    text = remove_emoji(text)
    text = clean_chars_and_contracts(text)
    
    tokens = tokenize.word_tokenize(text)
    tokens = remove_puntuation(tokens)

    return " ".join(tokens)

In [15]:
train_df.loc[:, "text_cleaned"] = train_df.text.apply(clean_text)
test_df.loc[:, "text_cleaned"] = test_df.text.apply(clean_text)

train_df.loc[:, "text_cleaned_v2"] = train_df.text_cleaned.apply(lemmatize_with_postag)
test_df.loc[:, "text_cleaned_v2"] = test_df.text_cleaned.apply(lemmatize_with_postag)

In [16]:
sample_df = train_df[["text", "text_cleaned", "text_cleaned_v2"]].sample(n=5)
for text, text_cleaned, text_cleaned_v2 in zip(sample_df.text, sample_df.text_cleaned, sample_df.text_cleaned_v2):
    print(f"Original:\n{text}")
    print(f"Cleaned:\n{text_cleaned}")
    print(f"Lemmatize:\n{text_cleaned_v2}")
    print("-"*25)

Original:
Photos: 17 people killed and over 25 injured in deadly Saudi Mosque suicide attack http://t.co/geEKnwJJSz
Cleaned:
Photos people killed and over injured in deadly Saudi Mosque suicide attack
Lemmatize:
Photos people kill and over injure in deadly Saudi Mosque suicide attack
-------------------------
Original:
Update: Bend FD says roofing co. workers accidentally cut through natural gas line in Post Office leading to evacuation for about a half-hr
Cleaned:
Update Bend FD says roofing workers accidentally cut through natural gas line in Post Office leading to evacuation for about a
Lemmatize:
Update Bend FD say roof worker accidentally cut through natural gas line in Post Office lead to evacuation for about a
-------------------------
Original:
.@karijobe and her band killed it tonight.  It was almost loud enough to drown out the tambourine behind me..... @codycarnes @AG_USA
Cleaned:
and her band killed it tonight It was almost loud enough to drown out the tambourine behind me


In [20]:
df = train_df["text_cleaned_v2"]

cv = CountVectorizer(stop_words="english")
X = cv.fit_transform(df)

vocab = cv.get_feature_names()

In [21]:
common_tweets = [tweet for tweet in train_df["text_cleaned_v2"].str.split()]

# Create a corpus from a list of texts
common_dictionary = Dictionary(common_tweets)
common_corpus = [common_dictionary.doc2bow(tweet) for tweet in common_tweets]

# Train the model on the corpus.
lda = gensim.models.LdaMulticore(common_corpus, num_topics=2, passes=50, workers=4)

In [24]:
for idx, topic in lda.print_topics():
    print("Topic: {} \nWords: {}".format(idx, topic))
    print("\n")

Topic: 0 
Words: 0.011*"9" + 0.010*"185" + 0.009*"165" + 0.008*"10" + 0.008*"26" + 0.006*"281" + 0.006*"820" + 0.005*"23" + 0.005*"35" + 0.005*"78"


Topic: 1 
Words: 0.038*"7" + 0.028*"10" + 0.025*"43" + 0.020*"35" + 0.019*"74" + 0.018*"26" + 0.016*"9" + 0.014*"75" + 0.010*"120" + 0.009*"164"




In [22]:
test_tweets = test_df["text_cleaned_v2"].str.split().values.tolist()
other_corpus = [common_dictionary.doc2bow(text) for text in test_tweets]

predictions = []
for oc in other_corpus:
    vector = lda[oc]
    y = sorted(vector, key=lambda tup: 1 - tup[1])[0][0]
    predictions.append(y)

In [26]:
def create_submission_file(predictions, ids, path, filename="submission.csv"):
    submission_data = {
        "id": ids,
        "target": predictions
    }

    submission_df = pd.DataFrame(submission_data)
    submission_df.to_csv(os.path.join(path, filename), index=False)
    
    print("Good luck!")

In [27]:
create_submission_file(predictions, test_df.id, output_path)

Good luck!
